In [12]:
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('../sdpm_py_util')
import ocn_functions as ocnfuns
import plotting_functions as pltfuns
import glob
from get_PFM_info import get_PFM_info
from datetime import datetime, timedelta
from netCDF4 import num2date, date2num, date2index


In [18]:

# Use the glob module to get a list of all files in the current directory
PFM=get_PFM_info()
fn = '/scratch/PFM_Simulations/LV1_Forecast/His/LV1_ocean_his.nc'

his_ds = netCDF4.Dataset(fn)
times = his_ds.variables['ocean_time']
times2 = num2date(times[:], times.units)

print(times2)




FileNotFoundError: [Errno 2] No such file or directory: b'/scratch/PFM_Simulations/LV1_Forecast/His/LV1_ocean_his.nc'

In [ ]:
t,eta = pltfuns.get_hisncs_tseries('zeta')